In [3]:
## utils 
def load_config(file_path):
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

from typing import Dict

def load_dataset(config: Dict, typ: str = "train", number_file: str = "001") -> pd.DataFrame:
    # Motornummer, Zeitschritt
    index_columns_names: List[str] =  ["UnitNumber","Cycle"]
    # 3 betriebsbereite Sensoreinstellungen
    operational_settings_columns_names: List[str] = ["Operation Setting "+str(i) for i in range(1,4)]
    # 21 Sensormessungen
    sensor_measure_columns_names: List[str] = ["Sensor Measure"+str(i) for i in range(1,22)]

    input_file_column_names: List[str] = index_columns_names + operational_settings_columns_names + sensor_measure_columns_names
    print(f"In total {len(input_file_column_names)} columns defined as {input_file_column_names}")

    ###############################
    ### load train data
    if typ == "train":
        type_path = config["dataloading"]["train_path"]
    elif typ == "test":
        type_path = config["dataloading"]["train_path"]
    elif "rul" in typ.lower():
        type_path = config["dataloading"]["RUL_path"]
    else:
        raise ValueError

    data: pd.DataFrame = pd.read_csv(config["dataloading"]["path_to_data"] + config["dataloading"]["train_path"] + number_file +'.txt', sep='\s+', header=None)
    data.columns = input_file_column_names
    return data


<>:31: SyntaxWarning: invalid escape sequence '\s'
<>:31: SyntaxWarning: invalid escape sequence '\s'
/var/folders/tq/88bplbbn4qgf097zd9c1g4dh0000gn/T/ipykernel_14124/2291215280.py:31: SyntaxWarning: invalid escape sequence '\s'
  data: pd.DataFrame = pd.read_csv(config["dataloading"]["path_to_data"] + config["dataloading"]["train_path"] + number_file +'.txt', sep='\s+', header=None)


In [2]:
###############################
## imports
###############################
import yaml
import os
import sys



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set path 
os.chdir("../")
pd.set_option('display.max_columns', 500)
print("Current working directory:", os.getcwd())

###############################
## load config
###############################
config_file_path = './configs/config.yaml'
config = load_config(config_file_path)

Current working directory: /Users/frbroy/Library/Mobile Documents/com~apple~CloudDocs/KIT/SoSe2024/PSDA/damage-propagation-modeling


In [9]:
###############################
## load data
###############################
train_data = load_dataset(config=config, typ="train", number_file="001")
train_data.head()

In total 26 columns defined as ['UnitNumber', 'Cycle', 'Operation Setting 1', 'Operation Setting 2', 'Operation Setting 3', 'Sensor Measure1', 'Sensor Measure2', 'Sensor Measure3', 'Sensor Measure4', 'Sensor Measure5', 'Sensor Measure6', 'Sensor Measure7', 'Sensor Measure8', 'Sensor Measure9', 'Sensor Measure10', 'Sensor Measure11', 'Sensor Measure12', 'Sensor Measure13', 'Sensor Measure14', 'Sensor Measure15', 'Sensor Measure16', 'Sensor Measure17', 'Sensor Measure18', 'Sensor Measure19', 'Sensor Measure20', 'Sensor Measure21']


,UnitNumber,Cycle,Operation Setting 1,Operation Setting 2,Operation Setting 3,Sensor Measure1,Sensor Measure2,Sensor Measure3,Sensor Measure4,Sensor Measure5,Sensor Measure6,Sensor Measure7,Sensor Measure8,Sensor Measure9,Sensor Measure10,Sensor Measure11,Sensor Measure12,Sensor Measure13,Sensor Measure14,Sensor Measure15,Sensor Measure16,Sensor Measure17,Sensor Measure18,Sensor Measure19,Sensor Measure20,Sensor Measure21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [80]:
def min_max(values):
    return values.max() - values.min()


In [81]:
window_size = 3
functs = ["mean", "sum", min_max]
# Compute the rolling mean for each column
rolling_mean = apply_rolling_window(train_data, window_size = window_size, functs=functs)

rolling_mean.head()

/var/folders/tq/88bplbbn4qgf097zd9c1g4dh0000gn/T/ipykernel_14124/2224889051.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rolling_data = df.groupby("UnitNumber").apply(lambda x: x.rolling(window=window_size).agg(agg_dict)).dropna()
/var/folders/tq/88bplbbn4qgf097zd9c1g4dh0000gn/T/ipykernel_14124/2224889051.py:11: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  rolling_data = rolling_data.drop(columns=["level_1"])


,UnitNumber,Operation Setting 1_mean,Operation Setting 1_sum,Operation Setting 1_min_max,Operation Setting 2_mean,Operation Setting 2_sum,Operation Setting 2_min_max,Operation Setting 3_mean,Operation Setting 3_sum,Operation Setting 3_min_max,Sensor Measure1_mean,Sensor Measure1_sum,Sensor Measure1_min_max,Sensor Measure2_mean,Sensor Measure2_sum,Sensor Measure2_min_max,Sensor Measure3_mean,Sensor Measure3_sum,Sensor Measure3_min_max,Sensor Measure4_mean,Sensor Measure4_sum,Sensor Measure4_min_max,Sensor Measure5_mean,Sensor Measure5_sum,Sensor Measure5_min_max,Sensor Measure6_mean,Sensor Measure6_sum,Sensor Measure6_min_max,Sensor Measure7_mean,Sensor Measure7_sum,Sensor Measure7_min_max,Sensor Measure8_mean,Sensor Measure8_sum,Sensor Measure8_min_max,Sensor Measure9_mean,Sensor Measure9_sum,Sensor Measure9_min_max,Sensor Measure10_mean,Sensor Measure10_sum,Sensor Measure10_min_max,Sensor Measure11_mean,Sensor Measure11_sum,Sensor Measure11_min_max,Sensor Measure12_mean,Sensor Measure12_sum,Sensor Measure12_min_max,Sensor Measure13_mean,Sensor Measure13_sum,Sensor Measure13_min_max,Sensor Measure14_mean,Sensor Measure14_sum,Sensor Measure14_min_max,Sensor Measure15_mean,Sensor Measure15_sum,Sensor Measure15_min_max,Sensor Measure16_mean,Sensor Measure16_sum,Sensor Measure16_min_max,Sensor Measure17_mean,Sensor Measure17_sum,Sensor Measure17_min_max,Sensor Measure18_mean,Sensor Measure18_sum,Sensor Measure18_min_max,Sensor Measure19_mean,Sensor Measure19_sum,Sensor Measure19_min_max,Sensor Measure20_mean,Sensor Measure20_sum,Sensor Measure20_min_max,Sensor Measure21_mean,Sensor Measure21_sum,Sensor Measure21_min_max,Cycle
0,1,-0.001033,-0.0031,0.0062,-0.000133,-0.0004,0.0007,100.0,300.0,0.0,518.67,1556.01,0.0,642.106667,1926.32,0.53,1589.836667,4769.51,3.83,1402.646667,4207.94,3.60,14.62,43.86,0.0,21.61,64.83,0.0,554.123333,1662.37,0.61,2388.060000,7164.18,0.04,9047.733333,27143.20,8.87,1.3,3.9,0.0,47.410000,142.23,0.22,522.120000,1566.36,0.76,2388.040000,7164.12,0.05,8134.446667,24403.34,7.13,8.423033,25.2691,0.0140,0.03,0.09,0.0,391.333333,1174.0,2.0,2388.0,7164.0,0.0,100.0,300.0,0.0,39.003333,117.01,0.11,23.395600,70.1868,0.0794,1
1,1,-0.000567,-0.0017,0.0062,0.000000,0.0000,0.0006,100.0,300.0,0.0,518.67,1556.01,0.0,642.283333,1926.85,0.20,1587.533333,4762.60,9.03,1403.070000,4209.21,2.33,14.62,43.86,0.0,21.61,64.83,0.0,554.153333,1662.46,0.70,2388.076667,7164.23,0.07,9048.830000,27146.49,8.87,1.3,3.9,0.0,47.296667,141.89,0.36,522.520000,1567.56,0.58,2388.060000,7164.18,0.05,8132.850000,24398.55,2.34,8.405933,25.2178,0.0636,0.03,0.09,0.0,391.333333,1174.0,2.0,2388.0,7164.0,0.0,100.0,300.0,0.0,38.943333,116.83,0.12,23.380567,70.1417,0.0794,2
2,1,-0.001833,-0.0055,0.0050,0.000033,0.0001,0.0005,100.0,300.0,0.0,518.67,1556.01,0.0,642.356667,1927.07,0.02,1584.543333,4753.63,5.20,1404.096667,4212.29,4.35,14.62,43.86,0.0,21.61,64.83,0.0,554.236667,1662.71,0.45,2388.083333,7164.25,0.05,9052.523333,27157.57,5.67,1.3,3.9,0.0,47.226667,141.68,0.15,522.490000,1567.47,0.67,2388.050000,7164.15,0.05,8133.620000,24400.86,0.60,8.405133,25.2154,0.0612,0.03,0.09,0.0,391.666667,1175.0,3.0,2388.0,7164.0,0.0,100.0,300.0,0.0,38.910000,116.73,0.07,23.374167,70.1225,0.0602,3
3,1,-0.001833,-0.0055,0.0050,-0.000100,-0.0003,0.0002,100.0,300.0,0.0,518.67,1556.01,0.0,642.273333,1926.82,0.27,1583.370000,4750.11,1.68,1402.153333,4206.46,7.85,14.62,43.86,0.0,21.61,64.83,0.0,554.373333,1663.12,0.67,2388.063333,7164.19,0.09,9051.436667,27154.31,5.67,1.3,3.9,0.0,47.190000,141.57,0.15,522.243333,1566.73,1.18,2388.050000,7164.15,0.05,8133.493333,24400.48,0.98,8.402800,25.2084,0.0612,0.03,0.09,0.0,392.000000,1176.0,2.0,2388.0,7164.0,0.0,100.0,300.0,0.0,38.920000,116.76,0.10,23.381733,70.1452,0.0375,4
4,1,-0.001733,-0.0052,0.0053,-0.000067,-0.0002,0.0003,100.0,300.0,0.0,518.67,1556.01,0.0,642.316667,1926.95,0.38,1586.546667,4759.64,9.47,1400.786667,4202.36,8.45,14.62,43.86,0.0,21.61,64.83,0.0,554.336667,1663.01,0.67,2388.033333,7164.10,0.04,9054.653333,27163.96,9

In [72]:
def apply_rolling_window(df, window_size, functs, exclude = ["UnitNumber", "Cycle"]):
    # Exclude "UnitNumber" from the aggregation dictionary
    
    agg_dict = {col: functs for col in df.columns if col not in exclude}
    
    # Apply rolling window and aggregation grouped by "UnitNumber"
    rolling_data = df.groupby("UnitNumber").apply(lambda x: x.rolling(window=window_size).agg(agg_dict)).dropna()
    
    # Merge back "UnitNumber" to the dataset
    rolling_data = rolling_data.reset_index()
    rolling_data = rolling_data.drop(columns=["level_1"])
    rolling_data["Cycle"] = df["Cycle"]

    ## refactor 2-levels of column names to only one level
    rolling_data.columns = ['_'.join(col).strip() if col[1] != "" else col[0] for col in rolling_data.columns.values]
    
    return rolling_data